## Normalizing and digitally separating H/E components

In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [7]:
# Creating a function of Normalization of H and E staining images

def norm_HnE(img, Io=240, alpha=1, beta=0.15):
        ######## Step 1: Convert RGB to OD ###################

    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
     
    ### reference maximum stain concentrations for H&E
    maxCRef = np.array([1.9705, 1.0308])
        
    h, w, c = img.shape
    img = img.reshape((-1,3))
        
        #calculating optical density
    OD = -np.log10((img.astype(np.float)+1)/Io) #Use this for opencv imread
        
        ############ Step 2: Remove data with OD intensity less than β ############
        # remove transparent pixels (clear region with no tissue)
    ODhat = OD[~np.any(OD < beta, axis=1)] 
        
        ############# Step 3: Calculate SVD on the OD tuples ######################
    eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))
        
        ######## Step 4: Create plane from the SVD directions with two largest values
        
    That = ODhat.dot(eigvecs[:,1:3]) #Dot product
        
        ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
    phi = np.arctan2(That[:,1],That[:,0])
        
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T
        
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
        
    
    Y = np.reshape(OD, (-1, 3)).T
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)
    
    # Separating H and E components
    
    H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
    H[H>255] = 254
    H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)
    
    
    E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
    E[E>255] = 254
    E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)
    
    return (Inorm, H, E)


In [8]:
from openslide import open_slide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff

In [ ]:
slide = open_slide("whole_slide_images/Normal Lymphnode.svs")

from openslide.deepzoom import DeepZoomGenerator

#Generate object for tiles using the DeepZoomGenerator
tiles = DeepZoomGenerator(slide, tile_size=256, overlap=0, limit_bounds=False)
#Here, we have divided our svs into tiles of size 256 with no overlap. 

#The tiles object also contains data at many levels. 
#To check the number of levels
print("The number of levels in the tiles object are: ", tiles.level_count)
print("The dimensions of data in each level are: ", tiles.level_dimensions)
#Total number of tiles in the tiles object
print("Total number of tiles = : ", tiles.tile_count)

###### processing and saving each tile to local directory
cols, rows = tiles.level_tiles[16]


orig_tile_dir_name = "whole_slide_images/original_tiles/"
norm_tile_dir_name = "whole_slide_images/normalized_tiles/"
H_tile_dir_name = "whole_slide_images/H_tiles/"
E_tile_dir_name = "whole_slide_images/E_tiles/"



In [ ]:
for row in range(rows):
    for col in range(cols):
        tile_name = str(col) + "_" + str(row)
        #tile_name = os.path.join(tile_dir, '%d_%d' % (col, row))
        #print("Now processing tile with title: ", tile_name)
        temp_tile = tiles.get_tile(16, (col, row))
        temp_tile_RGB = temp_tile.convert('RGB')
        temp_tile_np = np.array(temp_tile_RGB)
        #Save original tile
        tiff.imsave(orig_tile_dir_name+tile_name + "_original.tif", temp_tile_np)
        
        if temp_tile_np.mean() < 230 and temp_tile_np.std() > 15:
            print("Processing tile number:", tile_name)
            norm_img, H_img, E_img = norm_HnE(temp_tile_np, Io=240, alpha=1, beta=0.15)
        #Save the norm tile, H and E tiles      
            
            tiff.imsave(norm_tile_dir_name+tile_name + "_norm.tif", norm_img)
            tiff.imsave(H_tile_dir_name+tile_name + "_H.tif", H_img)
            tiff.imsave(E_tile_dir_name+tile_name + "_E.tif", E_img)
            
        else:
            print("NOT PROCESSING TILE:", tile_name)